In [6]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math

In [7]:
df_Market_Cap = pd.read_pickle("UsableData/MarketCap.pkl")
df_Employees = pd.read_pickle("UsableData/Employees.pkl")
df_Book_Value = pd.read_pickle("UsableData/BookValue.pkl")

In [21]:
df_eligible = df_Market_Cap
for column in df_eligible:
    df_eligible[column].astype("bool")
    df_eligible[column] = False
df_eligible.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1720 entries, 05P.D to ZZSG.DE^L07
Columns: 27 entries, 1995 to 2021
dtypes: bool(27)
memory usage: 123.3+ KB


In [23]:
for ric in df_Market_Cap.index:
    for year in df_Market_Cap.columns:
        mc = df_Market_Cap.at[ric, year]
        emp = df_Employees.at[ric, year]
        bv = df_Book_Value.at[ric, year]
        if(math.isnan(mc) or math.isnan(emp) or math.isnan(bv)):
            df_eligible.at[ric, year] = False
        else:
            df_eligible.at[ric, year] = True
df_eligible

Year,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
RIC,,,,,,,,,,,,,,,,,,,,,
05P.D,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0955.HK^D12,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
0FIBG.DE^F10,False,False,False,False,False,False,False,False,False,True,...,True,True,True,True,True,True,True,True,True,True
0RN6.L^G19,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
123F.DE,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZO1G.H,False,False,False,False,False,False,False,False,False,True,...,True,True,True,True,True,True,True,True,True,True
ZOOG.BE,False,False,False,False,False,True,True,True,True,True,...,True,True,False,False,False,False,False,False,False,False
ZPFGn.MU,False,False,False,False,False,True,True,True,True,True,...,True,True,False,False,False,False,False,True,True,True


In [32]:
df_eligible.describe()
for i in range(len(df_eligible.columns)):
    df = df_eligible[df_eligible[df_eligible.columns[i]] == True]
    print(f"{df_eligible.columns[i]}: {len(df.index)}")

1995: 51
1996: 107
1997: 205
1998: 346
1999: 659
2000: 837
2001: 904
2002: 949
2003: 1012
2004: 1069
2005: 1103
2006: 1143
2007: 1187
2008: 1206
2009: 1194
2010: 1205
2011: 1212
2012: 1216
2013: 1203
2014: 1130
2015: 1122
2016: 1107
2017: 1110
2018: 1137
2019: 1192
2020: 1229
2021: 1230
